In [72]:
import pandas as pd

In [73]:
df = pd.read_csv("nu.csv")
df_bb = pd.read_csv("bb.csv")

In [74]:
df.head()

,Data,Valor,Identificador,Descrição
0,01/11/2024,375.00,6724c561-1d42-4df6-a9b0-b0981071b9a2,Transferência Recebida - Gabriella Helena Aver...
1,02/11/2024,-111.98,67269c94-8bbb-43ea-ba9d-af579c6819da,Transferência enviada pelo Pix - Germano Franc...
2,03/11/2024,131.90,67278d39-1bf0-41ea-ab43-b271e780f844,Transferência Recebida - Francisco Diego Corre...
3,03/11/2024,-55.00,67278dc0-e4c1-4db4-a356-3cf836c9d01e,Transferência enviada pelo Pix - 55.136.070 FR...
4,04/11/2024,-1056.30,6728c8fe-3629-41ac-a2f8-ae6ff563c0f1,Transferência enviada pelo Pix - FACULDADES SA...


In [75]:
df.drop("Identificador", axis=1, inplace=True)

In [76]:
# date	description	type	amount
df.rename(
  columns={"Data": "date","Descrição": "description","Valor": "amount"}, 
  inplace=True
)

In [77]:
df_bb.rename(columns={
  'Data': 'date',
  'Valor': 'amount'
}, inplace=True)

df_bb['amount'] = df_bb['amount'].replace({r'"': '', r'\.': '', r',': '.'}, regex=True).astype(float)

df_bb['description'] = df_bb['Lancamento'] + ' - ' + df_bb['Detalhes']

df_bb = df_bb[['date', 'description', 'amount']]

In [78]:
df = pd.concat([df, df_bb], ignore_index=True)

In [79]:
df['source'] = pd.NA
df['categories'] = pd.NA

In [80]:
df["type"] = df["amount"].map(lambda value: "INCOME" if value > 0 else "OUTCOME")

In [81]:
df.head()

,date,amount,description,source,categories,type
0,01/11/2024,375.00,Transferência Recebida - Gabriella Helena Aver...,<NA>,<NA>,INCOME
1,02/11/2024,-111.98,Transferência enviada pelo Pix - Germano Franc...,<NA>,<NA>,OUTCOME
2,03/11/2024,131.90,Transferência Recebida - Francisco Diego Corre...,<NA>,<NA>,INCOME
3,03/11/2024,-55.00,Transferência enviada pelo Pix - 55.136.070 FR...,<NA>,<NA>,OUTCOME
4,04/11/2024,-1056.30,Transferência enviada pelo Pix - FACULDADES SA...,<NA>,<NA>,OUTCOME


In [82]:
mapperDf = pd.read_csv("mapper.tsv", sep="\t")
mapper = mapperDf.to_records().tolist()

In [83]:
def mapRow(row):
  for mapp in mapper:
    description = str(row["description"]).lower()
    amount = int(row['amount'])
    
    findKey = str(mapp[1]).lower()
    mapAmount = int(mapp[2])
    mapDescription = str(mapp[3])
    source = str(mapp[4])
    categories = str(mapp[5])

    if findKey not in description:
      continue
    
    if mapAmount != 0 and mapAmount != amount:
      continue

    row['description'] = mapDescription
    row['source'] = source
    row['categories'] = categories
    break

  if row['source'] == None or pd.isna(row['source']) or row['source'] == "":
    row['description'] = description.replace("transferência enviada pelo ", "").replace("compra no ", "")
    row['source'] = "CONTA CORRENTE"
    row['categories'] = "OUTROS"

  return row 

In [84]:
df = df.apply(mapRow, axis=1)
df.head()

,date,amount,description,source,categories,type
0,01/11/2024,375.00,transferência recebida - gabriella helena aver...,CONTA CORRENTE,OUTROS,INCOME
1,02/11/2024,-111.98,pix - germano francisco maciel soares - •••.02...,CONTA CORRENTE,OUTROS,OUTCOME
2,03/11/2024,131.90,Compensação Game Pass,DIEGO,COMPENSAÇÃO,INCOME
3,03/11/2024,-55.00,Compensação Game Pass,DIEGO,COMPENSAÇÃO,OUTCOME
4,04/11/2024,-1056.30,Faculdade,CONTA CORRENTE,ESTUDOS,OUTCOME


In [85]:
df = df[~df['description'].isin(["nan", "EXCLUIR"])]

In [86]:
df.to_csv(
  "./nu.tsv", 
  sep="\t", 
  lineterminator='\r\n', 
  na_rep="", 
  index=False,
  columns=["date", "description", "type", "amount", "source", "categories"],
  decimal=","
)